# Machine Learning Engineer Nanodegree
## Capstone Project - Starbucks app data

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
%matplotlib inline

In [14]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# Step 1 - Data preparation

## Portfolio

This dataset is pretty straightforward, we have a list of 10 offers with the specific informations.

_Data preparation:_
- Create a set of flags to split the _channels_ feature.
- Create a _channels_ counter.

In [15]:
rows, cols = portfolio.shape
print(f'There are {rows} rows and {cols} columns.\n')
portfolio

There are 10 rows and 6 columns.



,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [16]:
# Create a 0/1 flag for each channel
for channel in ['web', 'email', 'mobile', 'social']:
    portfolio[channel] = portfolio['channels'].apply(lambda x: 1 if channel in x else 0)
portfolio['channels_num'] = portfolio['channels'].apply(lambda x: len(x))

portfolio.drop('channels', 1, inplace=True)

portfolio

,reward,difficulty,duration,offer_type,id,web,email,mobile,social,channels_num
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,3
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,4
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0,3
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,3
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0,2
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,4
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,4
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1,3
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,4
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0,3


## Profile

This dataset has information about customers.

_Data preparation:_
- Transform the _became_member_on_ feature into a date feature.
- Analyse the missing features and eventually apply a transformation.

In [17]:
rows, cols = profile.shape
print(f'There are {rows} rows and {cols} columns.\n')
profile.head()

There are 17000 rows and 5 columns.



,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [18]:
cust = len(profile['id'].unique().tolist())
print(f'On {len(profile)} records we have {cust} customers')

On 17000 records we have 17000 customers


It seems that there are no duplicates.

In [ ]:
# To transform the feature, we create a new column
profile['member_on'] = pd.to_datetime(profile['became_member_on'].astype(str)).dt.date
profile.drop('became_member_on', 1, inplace=True)

There are some missing values in _gender_ and _income_: let's dive in this subset.

In [9]:
profile.loc[profile['gender'].isnull()].describe()

,age,income
count,2175.0,0.0
mean,118.0,NaN
std,0.0,NaN
min,118.0,NaN
25%,118.0,NaN
50%,118.0,NaN
75%,118.0,NaN
max,118.0,NaN


It seems like _income_ is always null and _age_ is always 118. Since we basically have no info on these profiles, we can safely drop them.

In [20]:
profile = profile.loc[profile['gender'].isnull() == False]

rows, cols = profile.shape
print(f'After data preparation, we have {rows} rows and {cols} columns.\n')
profile.head(10)

After data preparation, we have 14825 rows and 5 columns.



,gender,age,id,income,member_on
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017-05-09
5,M,68,e2127556f4f64592b11af22de27a7932,70000.0,2018-04-26
8,M,65,389bc3fa690240e798340f5a15918d5c,53000.0,2018-02-09
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017-11-11
13,F,61,aa4862eba776480b8bb9c68455b8c2e1,57000.0,2017-09-11
14,M,26,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,2014-02-13
15,F,62,31dda685af34476cad5bc968bdb01c53,71000.0,2016-02-11
16,M,49,62cf5e10845442329191fc246e7bcea3,52000.0,2014-11-13
18,M,57,6445de3b47274c759400cd68131d91b4,42000.0,2017-12-31


## Transcript

This dataframe contains the list of all actions on offers and transactions, with the relative information about money and time.

_Data preparation:_
- Analyse the _value_ feature and extract the information
- Reconstruct the customer journey, from receiving the offer to the (eventual) transaction

In [21]:
rows, cols = transcript.shape
print(f'There are {rows} rows and {cols} columns.\n')
transcript.head(10)

There are 306534 rows and 4 columns.



,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
5,389bc3fa690240e798340f5a15918d5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
6,c4863c7985cf408faee930f111475da3,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},0
8,aa4862eba776480b8bb9c68455b8c2e1,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
9,31dda685af34476cad5bc968bdb01c53,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0


In [22]:
keys = transcript.value.apply(lambda x: str(list(x.keys())))
print(keys.unique())

pd.crosstab(keys, transcript['event'])

["['offer id']" "['amount']" "['offer_id', 'reward']"]


event,offer completed,offer received,offer viewed,transaction
value,,,,
['amount'],0,0,0,138953
['offer id'],0,76277,57725,0
"['offer_id', 'reward']",33579,0,0,0


The value could be:
- **offer_id** for an *offer received* or *viewed*
- **reward** plus **offer_id** for an *offer completed*
- **amount** for a *transaction*

In [ ]:
# We create columns for each of these values, dropping the original column
for k in ['reward', 'amount']:
    transcript[k] = transcript['value'].apply(lambda x: x[k] if k in x else None)
    
transcript['offer_id'] = transcript['value'].apply(lambda x: x['offer_id'] if 'offer_id' in x else (x['offer id'] if 'offer id' in x else None))
transcript.drop('value', 1, inplace=True)

What is the history of a single customer? How many customers do we have? How many record has a customer?

In [24]:
cust_list = transcript.groupby('person')['time'].count()
cust_list.sort_values(ascending=False).head()

person
94de646f7b6041228ca7dec82adb97d2    51
8dbfa485249f409aa223a2130f40634a    49
79d9d4f86aca4bed9290350fb43817c2    48
d0a80415b84c4df4908b8403b19765e3    48
5e60c6aa3b834e44b822ea43a3efea26    48
Name: time, dtype: int64

The first customer has 51 records: let's take a look.

In [29]:
transcript.loc[transcript['person'] == cust_list.index[0]].sort_values('time')

,person,event,time,reward,amount,offer_id
55972,0009655768c64bdeb2e877511632db8f,offer received,168,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
77705,0009655768c64bdeb2e877511632db8f,offer viewed,192,NaN,NaN,5a8bc65990b245e5a138643cd4eb9837
89291,0009655768c64bdeb2e877511632db8f,transaction,228,NaN,22.16,None
113605,0009655768c64bdeb2e877511632db8f,offer received,336,NaN,NaN,3f207df678b143eea3cee63160fa8bed
139992,0009655768c64bdeb2e877511632db8f,offer viewed,372,NaN,NaN,3f207df678b143eea3cee63160fa8bed
153401,0009655768c64bdeb2e877511632db8f,offer received,408,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d
168412,0009655768c64bdeb2e877511632db8f,transaction,414,NaN,8.57,None
168413,0009655768c64bdeb2e877511632db8f,offer completed,414,5.0,NaN,f19421c1d4aa40978ebb69ca19b0e20d
187554,0009655768c64bdeb2e877511632db8f,offer viewed,456,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d
204340,0009655768c64bdeb2e877511632db8f,offer received,504,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4


It seems that for each *offer completed* there is a relative *transaction* with the same **time**. To verify this, we merge the to subsets.

In [30]:
tmp = pd.merge(transcript.loc[transcript['event'] == 'offer completed'],
               transcript.loc[transcript['event'] == 'transaction', ['person', 'event', 'time']],
               on=['person', 'time'])

print('There are {} offer completed.'.format(sum(transcript['event'] == 'offer completed')))
print('There are {} matches.'.format(tmp.shape[0]))

There are 33579 offer completed.
There are 33579 matches.


What's the relationship between *offer received* and *viewed*?

In [31]:
tmp = pd.merge(transcript.loc[transcript['event'] == 'offer viewed'].reset_index(),
               transcript.loc[transcript['event'] == 'offer received', ['person', 'time', 'offer_id']],
               on=['person', 'offer_id'])

print('There are {} offer viewed.'.format(sum(transcript['event'] == 'offer viewed')))
print('There are {} matches.'.format(len(tmp['index'].unique())))

There are 57725 offer viewed.
There are 57725 matches.


For each *viewed* offer there is a record with the relative *received* offer.

Finally, there's no clear relationship between *offer viewed* and *completed*: an offer can be viewed but not completed, and vice-versa (a customer completes an offer without knowing).



To better analyse the completion of the offers, we create a new dataframe recreating the entire funnel of customer's conversion:
- At first, we join the offer received with the eventual view by the customer
- Then, we follow the same approach to add information about conversion (*offer completed*)
- Finally, we add the *transaction*s. If there's a relative offer completed, we join the information to the row; if not, we add a new row.

In [35]:
# At first, let's join togheter the received offer with the relative (eventual) view
views = pd.merge(transcript.loc[transcript['event'] == 'offer received', ['person', 'offer_id', 'time']],
                 transcript.loc[transcript['event'] == 'offer viewed', ['person', 'offer_id', 'time']],
                 on=['person', 'offer_id'], how='left', suffixes=['_received', '_viewed'])

# We have to be sure to take the reception BEFORE the view (or when there's no view)
views = views.loc[(views['time_viewed'] >= views['time_received']) | (views['time_viewed'].isnull())]

# Then we take the NEAREST reception before the view
# We concatenate with the "missing view" dataframe portion, since aggregation results in a drop of these records
views = pd.concat((views.groupby(['person', 'offer_id', 'time_viewed']).agg({'time_received': 'max'}).reset_index(),
                  views.loc[views['time_viewed'].isnull()]),
                  axis=0, sort=True)

views.head()

,offer_id,person,time_received,time_viewed
0,3f207df678b143eea3cee63160fa8bed,0009655768c64bdeb2e877511632db8f,336,372.0
1,5a8bc65990b245e5a138643cd4eb9837,0009655768c64bdeb2e877511632db8f,168,192.0
2,f19421c1d4aa40978ebb69ca19b0e20d,0009655768c64bdeb2e877511632db8f,408,456.0
3,fafdcd668e3743c1bb461111dcafc2a4,0009655768c64bdeb2e877511632db8f,504,540.0
4,f19421c1d4aa40978ebb69ca19b0e20d,00116118485d4dfda04fdbaba9a87b5c,168,216.0


In [36]:
# Same reasoning joining the info about offer completion
comp = pd.merge(views,
                transcript.loc[transcript['event'] == 'offer completed', ['person', 'offer_id', 'time', 'reward']],
                on=['person', 'offer_id'], how='left').rename(columns={'time': 'time_completed'})

# When the completion time is AFTER the view time, we force the value to null
comp.loc[(comp['time_viewed'].isnull()) | (comp['time_viewed'] > comp['time_completed']), ['time_completed', 'reward']] = (np.nan, np.nan)
comp.drop_duplicates(inplace=True)

comp = pd.concat(
    (comp.groupby(['person', 'offer_id', 'time_completed', 'reward']).agg({'time_viewed': 'max', 'time_received': 'max'}).reset_index(),
     comp.loc[comp['time_completed'].isnull()]),
    axis=0, sort=True
)

comp.head()

,offer_id,person,reward,time_completed,time_received,time_viewed
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0011e0d4e6b944f998e987f904e8c1e5,5.0,576.0,408,432.0
1,2298d6c36e964ae4a3e7e9706d1fb8c2,0011e0d4e6b944f998e987f904e8c1e5,3.0,252.0,168,186.0
2,9b98b8c7a33c4b65b9aebfe6a799e6d9,0011e0d4e6b944f998e987f904e8c1e5,5.0,576.0,504,516.0
3,4d5c57ea9a6940dd891ad53e9dbe8da0,0020c2b971eb4e9188eac86d93036a77,10.0,510.0,408,426.0
4,fafdcd668e3743c1bb461111dcafc2a4,0020c2b971eb4e9188eac86d93036a77,2.0,54.0,0,12.0


In [37]:
# Now we join the information about the relative transaction
journey = pd.merge(comp,
                   transcript.loc[transcript['event'] == 'transaction', ['person', 'time', 'amount']],
                   left_on=['person', 'time_completed'], right_on=['person', 'time'], how='outer').rename(columns={'time': 'time_transaction'})
journey = journey[['person', 'offer_id', 'time_received', 'time_viewed', 'time_completed', 'time_transaction', 'amount', 'reward']]

rows, cols = journey.shape
print(f'There are {rows} rows and {cols} columns.\n')
journey.head()

There are 192096 rows and 8 columns.



,person,offer_id,time_received,time_viewed,time_completed,time_transaction,amount,reward
0,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,432.0,576.0,576.0,22.05,5.0
1,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504.0,516.0,576.0,576.0,22.05,5.0
2,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168.0,186.0,252.0,252.0,11.93,3.0
3,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,408.0,426.0,510.0,510.0,17.24,10.0
4,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,0.0,12.0,510.0,510.0,17.24,2.0


### Join data

Now we can join the 3 entities into one single dataset to analyse.

In [44]:
# Join offer info
tmp = pd.merge(journey,
               portfolio.rename(columns={'reward': 'pot_reward', 'id': 'offer_id'}),
               on='offer_id', how='left')

tmp.head()

,person,offer_id,time_received,time_viewed,time_completed,time_transaction,amount,reward,pot_reward,difficulty,duration,offer_type,web,email,mobile,social,channels_num
0,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,432.0,576.0,576.0,22.05,5.0,5.0,20.0,10.0,discount,1.0,1.0,0.0,0.0,2.0
1,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504.0,516.0,576.0,576.0,22.05,5.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0,3.0
2,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168.0,186.0,252.0,252.0,11.93,3.0,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,4.0
3,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,408.0,426.0,510.0,510.0,17.24,10.0,10.0,10.0,5.0,bogo,1.0,1.0,1.0,1.0,4.0
4,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,0.0,12.0,510.0,510.0,17.24,2.0,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0,4.0


In [46]:
# Join person info
# We will lose some records since we dropped some profiles with no informations

df = pd.merge(tmp,
              profile.rename(columns={'id': 'person'}),
              how='inner', on='person')

# To better visualize the journey, let's order by the offer received time / transaction time
df['time'] = df['time_received']
df.loc[df['time'].isnull(), 'time'] = df['time_transaction']
df.sort_values(['person', 'time', 'offer_id'], inplace=True)

rows, cols = df.shape
print(f'There are {rows} rows and {cols} columns.\n')
df.head()

There are 168645 rows and 22 columns.



,person,offer_id,time_received,time_viewed,time_completed,time_transaction,amount,reward,pot_reward,difficulty,duration,offer_type,web,email,mobile,social,channels_num,gender,age,income,member_on,time
133104,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168.0,192.0,NaN,NaN,NaN,NaN,0.0,0.0,3.0,informational,0.0,1.0,1.0,1.0,3.0,M,33,72000.0,2017-04-21,168.0
133108,0009655768c64bdeb2e877511632db8f,NaN,NaN,NaN,NaN,228.0,22.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,33,72000.0,2017-04-21,228.0
133103,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336.0,372.0,NaN,NaN,NaN,NaN,0.0,0.0,4.0,informational,1.0,1.0,1.0,0.0,3.0,M,33,72000.0,2017-04-21,336.0
133105,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408.0,456.0,NaN,NaN,NaN,NaN,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0,4.0,M,33,72000.0,2017-04-21,408.0
133109,0009655768c64bdeb2e877511632db8f,NaN,NaN,NaN,NaN,414.0,8.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,33,72000.0,2017-04-21,414.0


Our goal is to predict the right type of offer for each customer, in order to grow the **Conversion Rate** (the percentage of offers *completed* on all offers). What's the actual CR? We calculate it as **benchmark** of our model.

In [61]:
# Flag for offer viewed
tmp['flg_offer_viewed'] = 1
tmp.loc[tmp['time_viewed'].isnull(), 'flg_offer_viewed'] = 0

# Flag for transaction
tmp['flg_trans_offer'] = 0
tmp.loc[(tmp['time_transaction'].isnull() == False) & (tmp['time_received'].isnull() == False), 'flg_trans_offer'] = 1

# Flag for transaction without offer
tmp['flg_trans_no_offer'] = 0
tmp.loc[tmp['time_received'].isnull(), 'flg_trans_no_offer'] = 1

# Flag for offer without transaction
tmp['flg_offer_no_trans'] = 0
tmp.loc[tmp['time_transaction'].isnull(), 'flg_offer_no_trans'] = 1

tmp.head()

,person,offer_id,time_received,time_viewed,time_completed,time_transaction,amount,reward,pot_reward,difficulty,duration,offer_type,web,email,mobile,social,channels_num,flg_offer_viewed,flg_trans_offer,flg_trans_no_offer,flg_offer_no_trans
0,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,432.0,576.0,576.0,22.05,5.0,5.0,20.0,10.0,discount,1.0,1.0,0.0,0.0,2.0,1,1,0,0
1,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504.0,516.0,576.0,576.0,22.05,5.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0,3.0,1,1,0,0
2,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168.0,186.0,252.0,252.0,11.93,3.0,3.0,7.0,7.0,discount,1.0,1.0,1.0,1.0,4.0,1,1,0,0
3,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,408.0,426.0,510.0,510.0,17.24,10.0,10.0,10.0,5.0,bogo,1.0,1.0,1.0,1.0,4.0,1,1,0,0
4,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,0.0,12.0,510.0,510.0,17.24,2.0,2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0,4.0,1,1,0,0


In [68]:
print('CR on offers viewed')
display(tmp.loc[(tmp['flg_trans_no_offer'] == 0) & (tmp['flg_offer_viewed'] == 1)].groupby('offer_type').agg({'flg_trans_offer': ['mean', 'count']}))

print('\nCR on offers')
display(tmp.loc[tmp['flg_trans_no_offer'] == 0].groupby('offer_type').agg({'flg_trans_offer': ['mean', 'count']}))

CR on offers viewed


flg_trans_offer       
                         mean  count
offer_type                          
bogo                 0.433077  26762
discount             0.564103  23088
informational        0.000000  10831


CR on offers


flg_trans_offer       
                         mean  count
offer_type                          
bogo                 0.374766  30926
discount             0.417757  31176
informational        0.000000  14571

In [70]:
benchmark = tmp.copy()
benchmark['offer_amount'] = benchmark['amount'] * benchmark['flg_trans_offer']
benchmark['no_offer_amount'] = benchmark['amount'] * benchmark['flg_trans_no_offer']

benchmark.tail()

,person,offer_id,time_received,time_viewed,time_completed,time_transaction,amount,reward,pot_reward,difficulty,duration,offer_type,web,email,mobile,social,channels_num,flg_offer_viewed,flg_trans_offer,flg_trans_no_offer,flg_offer_no_trans,offer_amount,no_offer_amount
192091,b3a1272bc9904337b331bf348c3e8c17,NaN,NaN,NaN,NaN,714.0,1.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0,1.59
192092,68213b08d99a4ae1b0dcb72aebd9aa35,NaN,NaN,NaN,NaN,714.0,9.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0,9.53
192093,a00058cf10334a308c68e7631c529907,NaN,NaN,NaN,NaN,714.0,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0,3.61
192094,76ddbd6576844afe811f1a3c0fbb5bec,NaN,NaN,NaN,NaN,714.0,3.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0,3.53
192095,c02b10e8752c4d8e9b73f918558531f7,NaN,NaN,NaN,NaN,714.0,4.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0,4.05


In [72]:
benchmark = benchmark.groupby('person')['offer_amount', 'no_offer_amount'].sum()
benchmark['more_offer'] = 0
benchmark.loc[benchmark['offer_amount'] > benchmark['no_offer_amount'], 'more_offer'] = 1
benchmark['offer_amount_pct'] = benchmark['offer_amount'] / (benchmark['offer_amount'] + benchmark['no_offer_amount'])

benchmark.head()

,offer_amount,no_offer_amount,more_offer,offer_amount_pct
person,,,,
0009655768c64bdeb2e877511632db8f,0.00,127.60,0,0.000000
00116118485d4dfda04fdbaba9a87b5c,0.00,4.09,0,0.000000
0011e0d4e6b944f998e987f904e8c1e5,56.03,45.48,1,0.551965
0020c2b971eb4e9188eac86d93036a77,52.11,161.99,0,0.243391
0020ccbbb6d84e358d3414a3ff76cffd,36.50,117.55,0,0.236936


In [73]:
benchmark.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
offer_amount,17000.0,29.180687,61.577929,0.0,0.00,14.700000,41.930000,1754.29
no_offer_amount,17000.0,76.911258,101.053799,0.0,16.74,46.100000,107.502500,1245.12
more_offer,17000.0,0.116882,0.321289,0.0,0.00,0.000000,0.000000,1.00
offer_amount_pct,16578.0,0.229557,0.229602,0.0,0.00,0.192487,0.352845,1.00
